In [15]:
import torch
# import cvxpy as cp
import numpy as np
from itertools import combinations, product
from concurrent.futures import ThreadPoolExecutor
import math
from scipy.optimize import linprog

def construct_generator_matrix(n, k, P):
    """
    Constructs the systematic k×n generator matrix G given the k×(n−k) matrix P.
    """
    if P.shape != (k, n - k):
        raise ValueError(f"P must have shape ({k}, {n-k}), but got {P.shape}")
    
    I_k = torch.eye(k, dtype=P.dtype)
    G = torch.cat((I_k, P), dim=1)
    return G


def solve_m_height_lp(G, a, b, X, psi, debug=False):
    """
    Solves the m-height LP problem for a given tuple (a, b, X, psi).

    Parameters:
        G (numpy.ndarray): The k x n generator matrix.
        a (int): Index a in [n].
        b (int): Index b in [n] \ {a}.
        X (list): Subset X ⊆ [n] \ {a, b} with |X| = m - 1.
        psi (list): Binary vector of length m with elements in {-1, 1}.

    Returns:
        float: The optimal objective value of the LP problem.
               Returns 0 if infeasible, ∞ if unbounded.
    """
    k, n = G.shape
    Y = list(set(range(n)) - set(X) - {a, b})  # Y = [n] \ X \ {a, b}
    
    # Define the permutation τ
    tau = [a] + sorted(X) + [b] + sorted(Y)
    tau_inv = {tau[j]: j for j in range(n)}  # Inverse of τ
    
    # Objective function: maximize ∑(s0 * gi,a * ui)
    c = -psi[0] * G[:, a]  # Negative for maximization
    
    # Constraints
    A = []
    b_ub = []
    
    # Constraints for j ∈ X
    for j in X:
        j_tau = tau_inv[j]
        A.append(psi[j_tau] * G[:, j] - psi[0] * G[:, a])
        b_ub.append(0)
        A.append(-psi[j_tau] * G[:, j])
        b_ub.append(-1)
    
    # Equality constraint for j = b
    A_eq = [G[:, b]]
    b_eq = [1]
    
    # Constraints for j ∈ Y
    for j in Y:
        A.append(G[:, j])
        b_ub.append(1)
        A.append(-G[:, j])
        b_ub.append(1)
    
    # **Updated bounds to allow unrestricted variables**
    bounds = [-np.inf, np.inf]  

    # Solve the LP problem
    result = linprog(c, A_ub=A, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs-ds")
    
    if debug:
        print(f"n={n}, k={k}, a={a}, b={b}, X={X}, Y={Y}, tau={tau},\n tau_inv={tau_inv}")
        print(f"G={G},\n psi={psi},\n c={c},\n A={A},\n b_ub={b_ub},\n A_eq={A_eq},\n b_eq={b_eq}")
    if result.status == 2:  # Infeasible
        return 0
    elif result.status == 3:  # Unbounded
        return float('inf')
    else:
        return -result.fun  # Convert back to maximization value

from ortools.linear_solver import pywraplp
import numpy as np

def solve_m_height_lp_google(G, a, b, X, psi, debug=False):
    """
    Solves the m-height LP problem for a given tuple (a, b, X, psi) using Google's OR-Tools.
    
    Parameters:
        G (numpy.ndarray): The k x n generator matrix.
        a (int): Index a in [n].
        b (int): Index b in [n] \ {a}.
        X (list): Subset X ⊆ [n] \ {a, b} with |X| = m - 1.
        psi (list): Binary vector of length m with elements in {-1, 1}.
    
    Returns:
        float: The optimal objective value of the LP problem.
               Returns 0 if infeasible, ∞ if unbounded.
    """
    k, n = G.shape
    Y = list(set(range(n)) - set(X) - {a, b})  # Y = [n] \ X \ {a, b}
    
    # Define the permutation τ
    tau = [a] + sorted(X) + [b] + sorted(Y)
    tau_inv = {tau[j]: j for j in range(n)}  # Inverse of τ
    
    # Create the solver
    solver = pywraplp.Solver.CreateSolver('GLOP')
    if not solver:
        raise Exception("Solver not created. Ensure OR-Tools is installed correctly.")
    
    # Variables
    u = [solver.NumVar(-solver.infinity(), solver.infinity(), f'u_{i}') for i in range(k)]
    
    # Objective function: maximize ∑(s0 * gi,a * ui)
    objective = solver.Objective()
    for i in range(k):
        coefficient = float(psi[0] * G[i, a])  # Ensure the coefficient is a Python float
        objective.SetCoefficient(u[i], coefficient)  # Negative for maximization
    objective.SetMaximization()
    
    # Constraints
    # Constraints for j ∈ X
    for j in X:
        j_tau = tau_inv[j]
        constraint1 = solver.Constraint(-solver.infinity(), 0)
        constraint2 = solver.Constraint(-solver.infinity(), -1)
        for i in range(k):
            constraint1.SetCoefficient(u[i], float(psi[j_tau] * G[i, j] - psi[0] * G[i, a]))
            constraint2.SetCoefficient(u[i], float(-psi[j_tau] * G[i, j]))
    
    # Equality constraint for j = b
    eq_constraint = solver.Constraint(1, 1)
    for i in range(k):
        eq_constraint.SetCoefficient(u[i], float(G[i, b]))
    
    # Constraints for j ∈ Y
    for j in Y:
        constraint1 = solver.Constraint(-solver.infinity(), 1)
        constraint2 = solver.Constraint(-solver.infinity(), 1)
        for i in range(k):
            constraint1.SetCoefficient(u[i], float(G[i, j]))
            constraint2.SetCoefficient(u[i], float(-G[i, j]))
    
    # Solve the LP problem
    status = solver.Solve()
    
    if debug:
        print(f"n={n}, k={k}, a={a}, b={b}, X={X}, Y={Y}, tau={tau},\n tau_inv={tau_inv}")
        print(f"G={G},\n psi={psi}")
    
    if status == pywraplp.Solver.INFEASIBLE:
        return 0
    elif status == pywraplp.Solver.UNBOUNDED:
        return float('inf')
    else:
        return objective.Value()  # Return the maximized value

def compute_m_heights(G, lp_solve_function, max_m_value, debug=False):
    """
    Computes all m-heights of the code defined by the generator matrix G.
    
    Parameters:
        G (numpy.ndarray): The k x n generator matrix.
        lp_solve_function (function): Function to solve the LP problem.
        max_m_value (int): Maximum allowed value for m.
        debug (bool): If True, prints debug information.

    Returns:
        dict: Dictionary of m-heights.
    """
    start_time = time.time()

    n = G.shape[1]
    m_heights = {0: 1}
    num_tuples_expected = 0
    num_tuples_processed = 0

    # Restrict m based on max_m_value
    for m in range(1, min(max_m_value+1, n)):
        max_hm = float('-inf')

        for a, b in product(range(n), repeat=2):
            if a != b:
                for X in combinations(set(range(n)) - {a, b}, m - 1):
                    for psi in product([-1, 1], repeat=m):
                        num_tuples_processed += 1
                        
                        # Call the LP solver
                        result = lp_solve_function(G, a, b, list(X), list(psi), debug=debug)
                        max_hm = max(max_hm, result)

        m_heights[m] = max_hm
        num_tuples_expected += n * (n - 1) * math.comb(n - 2, m - 1) * (2 ** m)

        if max_hm == float('inf'):
            break

    # Fill remaining m-heights with infinity
    for m in range(len(m_heights), min(max_m_value+1, n)):
        m_heights[m] = float('inf')

    end_time = time.time()
    elapsed_time = end_time - start_time

    if debug:
        print(f"n = {n}, k = {G.shape[0]}")
        print(f"Expected number of tuples: {num_tuples_expected}")
        print(f"Processed {num_tuples_processed} tuples")
        print(f"Total computation time: {elapsed_time:.6f} seconds")

    return m_heights

def generate_random_P(n, k, existing_Ps):
    """
    Generates a unique random k x (n-k) matrix P with values bounded between -100 and 100.
    """
    while True:
        P = torch.randint(-100, 101, (k, n - k), dtype=torch.float32)
        if not any(torch.equal(P, existing) for existing in existing_Ps):
            existing_Ps.add(P)
            return P

def generate_dataset(n, k, m_values, num_samples):
    """
    Generates a dataset of random P matrices and computes all m-heights.
    """
    dataset = []
    existing_Ps = set()
    
    def process_sample(_):
        P = generate_random_P(n, k, existing_Ps)
        G = construct_generator_matrix(n, k, P)
        m_heights = compute_m_heights(G)
        for m in m_values:
            dataset.append((n, k, m, P.numpy().tolist(), m_heights.get(m, float('inf'))))
    
    with ThreadPoolExecutor() as executor:
        executor.map(process_sample, range(num_samples))
    
    return dataset




In [ ]:
# Example usage
n, k = 7, 4
m_values = [1, 2, 3]
num_samples = 10
dataset = generate_dataset(n, k, m_values, num_samples = 8)
print("Generated dataset:", dataset)

In [8]:
import time
matrices ={'G^6_9,3': torch.tensor([
    [0.992, -0.026, -0.019, -0.08, -0.487, 0.645, 0.473, 0.598, -0.732],
    [0.002, 0.969, 0.032, 0.291, -0.671, 0.208, -0.837, -0.398, 1.118],
    [-0.006, -0.013, 1.014, -0.805, 1.011, 1.518, -0.114, -0.347, -0.408]
])}
print("Computing m-heights using google solver...")
start_time = time.time()
for name, matrix in matrices.items():
    max_m_value = 6
    m_heights = compute_m_heights(matrix, lp_solve_function= solve_m_height_lp_google, max_m_value = max_m_value, debug=False)
    print(f"m-heights for {name}:")
    for m, height in m_heights.items():
        print(f"  {m}-height: {height}")
    print()
end_time = time.time()
print(f"Total computation time: {end_time - start_time:.6f} seconds")
print("Computing m-heights using scipy solver...")
start_time = time.time()
for name, matrix in matrices.items():
    max_m_value = 6
    m_heights = compute_m_heights(matrix, lp_solve_function= solve_m_height_lp, max_m_value = max_m_value, debug=False)
    print(f"m-heights for {name}:")
    for m, height in m_heights.items():
        print(f"  {m}-height: {height}")
    print()
end_time = time.time()
print(f"Total computation time: {end_time - start_time:.6f} seconds")

Computing m-heights using google solver...
m-heights for G^6_9,3:
  0-height: 1
  1-height: 2.3500656015145727
  2-height: 3.3650338956055244
  3-height: 4.2276606285236005
  4-height: 6.1288268148421
  5-height: 17.407621867212498
  6-height: 30.072372670600444
  7-height: inf
  8-height: inf

Total computation time: 157.062518 seconds
Computing m-heights using scipy solver...
m-heights for G^6_9,3:
  0-height: 1
  1-height: 2.3500656015145727
  2-height: 3.365033895605524
  3-height: 4.2276606285236005
  4-height: 6.128826814842115
  5-height: 17.40762186721249
  6-height: 30.072372670600423
  7-height: inf
  8-height: inf

Total computation time: 452.984297 seconds


In [ ]:
import torch

matrices = {
    "G^5_{9,4}": {
        "tensor": torch.tensor([
            [0.994, -0.014, 0.006, 0.024, -0.32, 0.427, -0.414, -0.897, 0.429],
            [-0.02, 1.022, -0.006, 0.02, 0.649, -0.593, 0.469, 0.595, 0.16],
            [-0.006, -0.035, 1.017, 0.019, -0.233, -1.258, 0.611, 0.422, 0.094],
            [0.023, 0.013, 0.011, 1.028, 0.45, 0.82, -1.787, -1.595, 0.438]
        ], dtype=torch.float),
        "M_value": 5,
        "shape": (4, 9)  # 4 rows, 9 columns
    },
    
    "G^3_{9,5}": {
        "tensor": torch.tensor([
            [1.0, 0.0, 0.0, 0.0, 0.0, -0.761, 0.46, -0.524, 0.79],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.487, -0.86, 1.421, -0.091],
            [0.0, 0.0, 1.0, 0.0, 0.0, -0.789, 0.552, -1.035, -0.935],
            [0.0, 0.0, 0.0, 1.0, 0.0, -0.353, -0.764, 0.075, -0.781],
            [0.0, 0.0, 0.0, 0.0, 1.0, -0.147, -1.116, 0.873, -0.328]
        ], dtype=torch.float),
        "M_value": 4,
        "shape": (5, 9)  # 5 rows, 9 columns
    },
    
    "G^2_{9,6}": {
        "tensor": torch.tensor([
            [-0.553, -0.817, 0.513, -0.784, -0.215, -0.784, -0.784, -0.215, 0.513],
            [-0.215, -0.215, -0.553, -0.553, 0.513, 0.513, -0.215, -0.553, -0.215],
            [-0.817, -0.784, 0.293, -0.215, -0.817, -0.553, 0.513, 0.513, 0.293],
            [0.513, -0.553, -0.817, 0.513, 0.293, -0.215, -0.817, 0.293, -0.784],
            [-0.784, 0.513, -0.215, -0.817, -0.784, 0.293, 0.293, -0.784, -0.553],
            [0.293, 0.293, -0.784, 0.293, -0.553, -0.553, -0.817, -0.553, -0.817]
        ], dtype=torch.float),
        "M_value": 3,
        "shape": (6, 9)  # 6 rows, 9 columns
    },
    
    "G^6_{10,4}": {
        "tensor": torch.tensor([
            [1.016, -0.021, -0.006, -0.008, 1.22, 0.271, 0.06, -0.756, -1.301, -1.257],
            [-0.008, 1.001, -0.001, 0.021, -1.037, 0.024, -0.321, 1.061, 0.412, -0.362],
            [-0.006, 0.013, 0.988, -0.015, 0.452, -0.168, -0.781, 0.677, -0.189, -0.171],
            [-0.03, 0.022, -0.008, 0.986, 0.007, 2.065, -2.632, 0.765, 0.137, 0.908]
        ], dtype=torch.float),
        "M_value": 6,
        "shape": (4, 10)  # 4 rows, 10 columns
    },
    
    "G^2_{10,5}": {
        "tensor": torch.tensor([
            [0.005, 0.992, 0.005, -1.276, 0.005, 1.974, 1.974, -1.276, -1.276, -0.033],
            [1.974, 1.974, -0.033, 0.005, -0.033, -0.033, -1.276, 0.992, 0.992, -1.276],
            [-0.033, -1.276, 1.974, 0.992, 1.974, 0.992, 0.992, 1.974, 0.005, 1.974],
            [0.992, 0.005, 0.992, -0.033, -1.276, -1.276, -0.033, -0.033, 1.974, 0.992],
            [-1.276, -0.033, -1.276, 1.974, 0.992, 0.005, 0.005, 0.005, -0.033, 0.005]
        ], dtype=torch.float),
        "M_value": 5,
        "shape": (5, 10)  # 5 rows, 10 columns
    },
    
    "G^3_{10,6}": {
        "tensor": torch.tensor([
            [1.043, 0.569, 0.569, 0.023, 0.023, 1.196, 0.569, 1.043, -0.201, 1.196],
            [-0.201, -0.201, -0.201, -0.201, 1.196, -1.141, -1.141, 1.196, 1.043, -0.201],
            [1.196, -1.141, -1.141, 1.043, 0.569, 0.023, -0.201, -1.141, 1.196, 0.023],
            [0.023, 1.196, 0.023, 1.196, -0.201, 1.043, 1.196, 0.023, 0.569, -1.141],
            [0.569, 1.043, 1.043, -1.141, -1.141, -0.201, 0.023, 0.569, 0.023, 1.043],
            [-1.141, 0.023, 1.196, 0.569, 1.043, 0.569, 1.043, -0.201, -1.141, 0.569]
        ], dtype=torch.float),
        "M_value": 4,
        "shape": (6, 10)  # 6 rows, 10 columns
    }
}

In [22]:
for name, matrix in matrices.items():
    # max_m_value = 6
    start_time = time.time()
    m_heights = compute_m_heights(matrix['tensor'], lp_solve_function= solve_m_height_lp_google, max_m_value = matrix['M_value'], debug=False)
    end_time = time.time()
    print(f"Total computation time for {name}: {end_time - start_time:.6f} seconds")
    print(f"m-heights for {name}:")
    for m, height in m_heights.items():
        print(f"  {m}-height: {height}")
    print()

Total computation time for G^3_{9,4}: 85.485159 seconds
m-heights for G^3_{9,4}:
  0-height: 1
  1-height: 3.1914711467882766
  2-height: 4.860550214039052
  3-height: 7.594460756314859
  4-height: 40.43413856078109
  5-height: 101.40563396209151

Total computation time for G^3_{9,5}: 36.614604 seconds
m-heights for G^3_{9,5}:
  0-height: 1
  1-height: 3.258875335866644
  2-height: 4.9621324299025975
  3-height: 12.503611926317069
  4-height: 2769.5462546257118

Total computation time for G^5_{9,6}: 10.755044 seconds
m-heights for G^5_{9,6}:
  0-height: 1
  1-height: 4.491732329998033
  2-height: 26.680362946453673
  3-height: 163.40954659332357

Total computation time for G^6_{10,4}: 465.452593 seconds
m-heights for G^6_{10,4}:
  0-height: 1
  1-height: 2.3129255307399657
  2-height: 3.403928923825311
  3-height: 8.383864727852032
  4-height: 14.22269707942369
  5-height: 49.37241444249539
  6-height: 322.44334298968573

Total computation time for G^7_{10,5}: 266.390098 seconds
m-heig

In [16]:
# Define the generator matrix G
G = torch.tensor([
    [1, 0, 0.4759809, 0.9938236, 0.819425],
    [0, 1, -0.8960798, -0.7442706, 0.3345122]
], dtype=torch.double)
# k,n = G.shape
# for m in range(
G_2_8_3 = torch.tensor([
    [1.0, 0.0, 0.0, -0.254, 0.511, -0.749, -0.73, -0.73],
    [0.0, 1.0, 0.0, -1.536, -0.006, 0.174, 1.339, 1.339],
    [0.0, 0.0, 1.0, -0.624, 0.705, 1.01, 0.943, 0.943]
], dtype=torch.float32)

# Compute the m-heights for m = 1, 2, 3, 4
m_heights = compute_m_heights(G_2_8_3, lp_solve_function= solve_m_height_lp_google, max_m_value=2, debug=False)

# Print the results
for m in range(0, len(m_heights)):
    print(f"{m}-height: {m_heights.get(m, float('inf'))}")

0-height: 1
1-height: 2.0487317504667724
2-height: 2.4839415001033394


In [18]:
G_3_8_3 = torch.tensor([
    [1.0, 0.0, 0.0, 1.157, 0.017, 1.164, 1.353, 1.353],
    [0.0, 1.0, 0.0, -0.076, -0.741, -0.888, -1.495, -1.495],
    [0.0, 0.0, 1.0, 1.075, -0.702, 0.876, -0.266, -0.266]
], dtype=torch.float32)

# Compute the m-heights for m = 1, 2, 3, 4
m_heights = compute_m_heights(G_3_8_3, lp_solve_function= solve_m_height_lp_google, max_m_value=3, debug=False)

# Print the results
for m in range(0, len(m_heights)):
    print(f"{m}-height: {m_heights.get(m, float('inf'))}")

0-height: 1
1-height: 1.8900654122787162
2-height: 2.437295966294273
3-height: 3.7061633468314055


In [19]:
G_6_3 = torch.tensor([
    [1.0, 0.0, 0.0, 0.7, -0.791, 0.043],
    [0.0, 1.0, 0.0, -0.647, 0.833, 1.162],
    [0.0, 0.0, 1.0, 0.311, 1.284, 1.028]
], dtype=torch.float32)

# Compute the m-heights for m = 1, 2, 3, 4
m_heights = compute_m_heights(G_6_3, lp_solve_function= solve_m_height_lp_google, max_m_value=2, debug=False)

# Print the results
for m in range(0, len(m_heights)):
    print(f"{m}-height: {m_heights.get(m, float('inf'))}")

0-height: 1
1-height: 2.1107468992031593
2-height: 2.8673200357539312


In [ ]:
# Define the matrices from the image
matrices = {
    "G_6_3": torch.tensor([
        [1.0, 0.0, 0.0, 0.7, -0.791, 0.043],
        [0.0, 1.0, 0.0, -0.647, 0.833, 1.162],
        [0.0, 0.0, 1.0, 0.311, 1.284, 1.028]
    ], dtype=torch.float32),
    "G_7_3": torch.tensor([
        [1.087, -0.085, -0.087, 1.134, 0.5, 0.19, 0.955],
        [-0.037, 0.977, -0.033, -0.323, -0.315, 0.635, 0.394],
        [-0.109, 0.035, 1.039, 0.527, -1.093, 0.751, 0.159]
    ], dtype=torch.float32),
    "G_8_2": torch.tensor([ # Only this is wrong
        [1.0, 0.0, 0.534, 0.583, -0.709, -0.533, 0.543, 0.927],
        [0.0, 1.0, 0.611, -1.211, 0.233, -0.784, -0.659, -0.947]
    ], dtype=torch.float32),
    "G_8_3": torch.tensor([
        [1.0, 0.0, 0.0, -0.254, 0.511, -0.749, -0.73, -0.73],
        [0.0, 1.0, 0.0, -1.536, -0.006, 0.174, 1.339, 1.339],
        [0.0, 0.0, 1.0, -0.624, 0.705, 1.01, 0.943, 0.943]
    ], dtype=torch.float32)
}

# Calculate m-heights for each matrix
for name, matrix in matrices.items():
    m_heights = compute_m_heights(matrix, debug=False)
    print(f"m-heights for {name}:")
    for m, height in m_heights.items():
        print(f"  {m}-height: {height}")
    print()

n =6, k = 3
Expected number of tuples : 3900
Processed 3900 tuples
m-heights for G_6_3:
  0-height: 1
  1-height: 2.1107468992031593
  2-height: 2.8673200357539312
  3-height: 74.18604255061089
  4-height: inf
  5-height: inf

n =7, k = 3
Expected number of tuples : 17724
Processed 17724 tuples
m-heights for G_7_3:
  0-height: 1
  1-height: 2.3385431386722395
  2-height: 2.941981129663437
  3-height: 6.038029309675049
  4-height: 10.132843146161273
  5-height: inf
  6-height: inf

n =8, k = 2
Expected number of tuples : 81648
Processed 81648 tuples
m-heights for G_8_2:
  0-height: 1
  1-height: 1.4055812685827276
  2-height: 1.583720838865125
  3-height: 1.8740000128746033
  4-height: 2.6840797290630367
  5-height: 4.19725419969681
  6-height: 24.771913345443416
  7-height: 42.36926161949583

n =8, k = 3
Expected number of tuples : 52976
Processed 52976 tuples
m-heights for G_8_3:
  0-height: 1
  1-height: 2.0487317504667724
  2-height: 2.4839415001033394
  3-height: 3.573830481906119


In [66]:
# for a, b in combinations(range(5), 2):
matrices = {
    "G5_2": torch.tensor([
        [0.911, 0.03, 1.481, -0.756, 1.249],
        [-0.049, 0.975, 1.511, -1.303, 0.74]
    ]),
    "G3_5_2": torch.tensor([
        [0.909, 0.014, 1.493, -0.738, 1.235],
        [-0.039, 0.967, 1.519, -1.299, 0.719]
    ]),
    "G6_2": torch.tensor([
        [0.859, -0.103, 0.746, -0.35, 0.892, -0.95],
        [0.025, 1.063, -1.591, -0.721, -1.115, 0.679]
    ]),
    "G4_6_2": torch.tensor([
        [0.922, -0.121, 0.746, -0.368, 0.896, -1.002],
        [0.022, 1.069, -1.593, -0.702, -1.089, 0.676]
    ]),
    "G6_3": torch.tensor([
        [0.236, -1.136, -0.45, -1.136, 0.236, -0.45],
        [-1.136, -0.45, 0.236, 0.236, -0.45, -1.136],
        [-0.45, 0.236, -1.136, -0.45, -1.136, 0.236]
    ]),
    "G7_2": torch.tensor([
        [1.0, 0.0, -0.74, -1.09, 0.463, 0.584, 0.719],
        [0.0, 1.0, 1.106, 0.79, 0.713, 0.706, -0.531]
    ])
}
for name, matrix in matrices.items():
    m_heights = compute_m_heights(matrix, debug=False)
    print(f"m-heights for {name}:")
    for m, height in m_heights.items():
        print(f"  {m}-height: {height}")
    print()

n =5, k = 2
Expected number of tuples : 1080
Processed 1080 tuples
m-heights for G5_2:
  0-height: 1
  1-height: 1.4781981965594937
  2-height: 1.834649181221645
  3-height: 3.394813000095867
  4-height: 17.636909514341095

n =5, k = 2
Expected number of tuples : 1080
Processed 1080 tuples
m-heights for G3_5_2:
  0-height: 1
  1-height: 1.5097039203110674
  2-height: 1.8789077400510568
  3-height: 3.2536682913821378
  4-height: 53.19979767955847

n =6, k = 2
Expected number of tuples : 4860
Processed 4860 tuples
m-heights for G6_2:
  0-height: 1
  1-height: 1.4597774344977115
  2-height: 1.7385023808030382
  3-height: 2.275777038777704
  4-height: 4.312301164221178
  5-height: 25.061398433986742

n =6, k = 2
Expected number of tuples : 4860
Processed 4860 tuples
m-heights for G4_6_2:
  0-height: 1
  1-height: 1.4764816470568263
  2-height: 1.7148883259020211
  3-height: 2.4357335573455607
  4-height: 4.0902136218143195
  5-height: 607.6528320030025

n =6, k = 3
Expected number of tuple

In [67]:
matrices = {
    'G^3_8,3': torch.tensor([
        [1.0, 0.0, 0.0, 1.157, 0.017, 1.164, 1.353, 1.353],
        [0.0, 1.0, 0.0, -0.076, -0.741, -0.888, -1.495, -1.495],
        [0.0, 0.0, 1.0, 1.075, -0.702, 0.876, -0.266, -0.266]
    ]),
    'G^4_8,3': torch.tensor([
        [1.049, 0.026, 0.021, 0.893, 0.809, 0.27, -1.151, 1.402],
        [-0.008, 1.112, 0.022, 0.52, -1.689, -2.2, 0.285, 0.396],
        [-0.076, 0.101, 0.996, -0.529, -1.262, 1.039, -0.979, 0.582]
    ]),
    'G^5_8,3': torch.tensor([
        [1.042, 0.06, 0.036, 0.887, 0.807, 0.269, -1.179, 1.419],
        [0.006, 1.086, 0.036, 0.506, -1.693, -2.209, 0.303, 0.394],
        [-0.076, 0.092, 0.987, -0.541, -1.271, 1.042, -0.99, 0.574]
    ]),
    'G^4_9,2': torch.tensor([
        [1.0, 0.0, 0.0, 0.476, 0.476, 0.994, 0.994, 0.819, 0.819],
        [0.0, 1.0, 1.0, -0.896, -0.896, -0.744, -0.744, 0.335, 0.335]
    ]),
    'G^5_9,2': torch.tensor([
        [1.0, 0.0, 0.865, 0.111, -1.464, 1.522, 0.364, 0.511, -1.228],
        [0.0, 1.0, 0.168, 0.916, 0.509, -0.935, 0.776, -0.863, 1.517]
    ]),
    'G^6_9,2': torch.tensor([
               [1.0, 0.0, 0.0, 1.586, 1.586, -0.655, -0.655, 1.193, 1.193],
        [0.0, 1.0, 1.0, 1.555, 1.555, -1.244, -1.244, 0.684, 0.684]
    ]),
    'G^3_9,3': torch.tensor([
        [1.0, 0.0, 0.0, -0.298, -0.954, 1.01, -0.104, -0.145, 0.893],
        [0.0, 1.0, 0.0, 0.004, 0.784, -1.751, -0.882, -0.802, -1.202],
        [0.0, 0.0, 1.0, -0.911, 0.346, 0.841, 0.655, 0.613, -0.594]
    ]),
    'G^4_9,3': torch.tensor([
        [1.003, 0.003, -0.011, -0.073, -0.461, 0.676, 0.485, 0.631, -0.72],
        [0.021, 0.999, 0.018, 0.269, -0.649, 0.202, -0.858, -0.421, 1.135],
        [0.013, 0.003, 1.009, -0.764, 1.034, 1.532, -0.095, -0.384, -0.392]
    ]),
    'G^5_9,3': torch.tensor([
        [0.992, -0.02, 0.009, -1.579, -0.204, -0.952, 0.121, 0.519, -1.571],
        [-0.014, 0.989, 0.005, 1.215, -1.134, -0.922, -0.475, -0.331, 0.192],
        [-0.004, 0.005, 0.98, 0.052, 1.081, -0.457, 0.69, -0.476, -0.885]
    ]),
    'G^6_9,3': torch.tensor([
        [0.992, -0.026, -0.019, -0.08, -0.487, 0.645, 0.473, 0.598, -0.732],
        [0.002, 0.969, 0.032, 0.291, -0.671, 0.208, -0.837, -0.398, 1.118],
        [-0.006, -0.013, 1.014, -0.805, 1.011, 1.518, -0.114, -0.347, -0.408]
    ])}
for name, matrix in matrices.items():
    m_heights = compute_m_heights(matrix, debug=False)
    print(f"m-heights for {name}:")
    for m, height in m_heights.items():
        print(f"  {m}-height: {height}")
    print()

n =8, k = 3
Expected number of tuples : 52976
Processed 52976 tuples
m-heights for G^3_8,3:
  0-height: 1
  1-height: 1.8900654122787162
  2-height: 2.437295966294273
  3-height: 3.7061633468314055
  4-height: 25.99982490195746
  5-height: inf
  6-height: inf
  7-height: inf

n =8, k = 3
Expected number of tuples : 74480
Processed 74480 tuples
m-heights for G^4_8,3:
  0-height: 1
  1-height: 3.3623128505267776
  2-height: 3.933094511693524
  3-height: 8.118027835807137
  4-height: 9.024239379486158
  5-height: 21.974611660704827
  6-height: inf
  7-height: inf

n =8, k = 3
Expected number of tuples : 74480
Processed 74480 tuples
m-heights for G^5_8,3:
  0-height: 1
  1-height: 3.403286363684166
  2-height: 4.031600744956478
  3-height: 7.856864804684318
  4-height: 9.540059401297862
  5-height: 20.394235547639468
  6-height: inf
  7-height: inf

n =9, k = 2
Expected number of tuples : 314928
Processed 314928 tuples
m-heights for G^4_9,2:
  0-height: 1
  1-height: 1.1450389656046391
  2

In [26]:
import sys
import pickle
import gzip
import random
def read_dataset(file_path):
    """
    Reads a generator matrix dataset file, prints n, k, and the number of samples,
    and returns a random sample from the dataset.
    """
    try:
        # Load the dataset from the compressed file
        with gzip.open(file_path, 'rb') as f:
            dataset = pickle.load(f)
        
        # Ensure the dataset is not empty
        if not dataset:
            print("The dataset is empty.")
            return None

        # Extract n and k from the first sample
        first_sample = dataset[0]
        generator_matrix = first_sample["generator_matrix"]
        n = len(generator_matrix[0])  # Number of columns
        k = len(generator_matrix)    # Number of rows

        # Print dataset details
        print(f"Dataset loaded from: {file_path}")
        print(f"n (columns): {n}")
        print(f"k (rows): {k}")
        print(f"Number of samples in the dataset: {len(dataset)}")

        # Pick a random sample
        random_sample = random.choice(dataset)
        print("Random sample selected.")
        return random_sample

    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [27]:
random_sample = read_dataset(file_path= "generator_matrices\G_9_4_maxM5.pkl.gz")

Dataset loaded from: generator_matrices\G_9_4_maxM5.pkl.gz
n (columns): 9
k (rows): 4
Number of samples in the dataset: 83
Random sample selected.


In [28]:
print(random_sample)

{'generator_matrix': [[1.0, 0.0, 0.0, 0.0, -99.62687683105469, 79.57284545898438, -78.35952758789062, -53.729225158691406, -5.206092834472656], [0.0, 1.0, 0.0, 0.0, 79.83384704589844, -72.63645935058594, -83.50702667236328, 84.11811828613281, -82.01593017578125], [0.0, 0.0, 1.0, 0.0, -18.56578826904297, -14.970138549804688, -7.652069091796875, -23.089279174804688, -5.3660125732421875], [0.0, 0.0, 0.0, 1.0, -69.23001098632812, -96.98092651367188, 59.25172424316406, 28.46734619140625, 9.793449401855469]], 'max_m_value': 5}
